In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt


In [2]:

# Qiskit imports
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import ZZFeatureMap, TwoLocal
from qiskit_machine_learning.algorithms import VQC, QSVC
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Session
from qiskit.primitives import StatevectorSampler
from qiskit_algorithms.state_fidelities import ComputeUncompute

In [9]:
# For local simulation
from qiskit_aer import AerSimulator

## Load Pima Indian Dataset

In [10]:
df = pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [11]:
data = df.to_numpy()
X = data[:, :-1]  # Use first 8 features to match Pima structure
y = data[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Scale to match Pima-like ranges
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(X_train.shape, X_test.shape)


(576, 8) (192, 8)


## Use PCA to select most significant features

In [12]:
from sklearn.decomposition import PCA

pca = PCA()

X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

## Run classical SVM for comparison

In [13]:
n_features = 4

print("Running Classical SVM...")

# Train classical SVM
classical_svm = SVC(kernel='rbf', C=1.0, random_state=1)
classical_svm.fit(X_train_pca[:,:n_features], y_train)

y_pred_train_classical = classical_svm.predict(X_train_pca[:, :n_features])
accuracy_train_classical = accuracy_score(y_train, y_pred_train_classical)
# Predict and evaluate
y_pred_classical = classical_svm.predict(X_test_pca[:,:n_features])
accuracy_classical = accuracy_score(y_test, y_pred_classical)

print(f"Classical SVM Accuracy on Training Data: {accuracy_train_classical: .3f}")
print(f"Classical SVM Accuracy on Test Data: {accuracy_classical:.3f}")


Running Classical SVM...
Classical SVM Accuracy on Training Data:  0.786
Classical SVM Accuracy on Test Data: 0.688


## Set Up IBM Quantum

Setup IBM Quantum Cloud connection using qiskit-ibm-runtime

In [28]:
print("\nSetting up quantum backend...")

token = "VMgmA9Gjyklr5uAIq8S6FYXrmKvoVN3QLo714I6KjRBA"
instance_id = "QML"

service = QiskitRuntimeService(channel="ibm_cloud", token=token, instance=instance_id)

# Get available backends
backends = service.backends(simulator=False, operational=True)
print("Available quantum backends:")
for backend in backends:
    print(f"  - {backend.name}: {backend.num_qubits} qubits")

# Select least busy real quantum computer
if backends:
    backend = service.least_busy(operational=True, simulator=False)
    print(f"\nSelected backend: {backend.name}")
else:
    print("No quantum backends available, using simulator")
    backend = service.backend('ibmq_qasm_simulator')
    



Setting up quantum backend...
Available quantum backends:
  - ibm_brisbane: 127 qubits
  - ibm_torino: 133 qubits

Selected backend: ibm_brisbane


## Create quantum kernel for QSVM

In [29]:
import qiskit_machine_learning
print(qiskit_machine_learning.__version__)

0.8.3


In [30]:
# Feature map - encodes classical data into quantum states
feature_map = ZZFeatureMap(feature_dimension=n_features, 
                            reps=4, 
                            entanglement='linear')



In [31]:
from qiskit.primitives import BackendSampler
# Create a sampler with the Brisbane backend

# Create a V1 compatible sampler
sampler = BackendSampler(backend)

# Create fidelity and kernel
fidelity = ComputeUncompute(sampler=sampler)

quantum_kernel = FidelityQuantumKernel(feature_map=feature_map, fidelity=fidelity)


C:\Users\leqian\AppData\Local\Temp\ipykernel_14852\2841231180.py:5: DeprecationWarning: The class ``qiskit.primitives.backend_sampler.BackendSampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `BackendSampler` class is `BackendSamplerV2`.
  sampler = BackendSampler(backend)


## Run Quantum SVM

In [ ]:
print("Running Quantum SVM...")


# Create QSVC with quantum kernel
qsvc = QSVC(quantum_kernel=quantum_kernel)

# Train quantum SVM
print("Training quantum model...")
qsvc.fit(X_train_pca[:,:n_features], y_train)

Running Quantum SVM...
Training quantum model...


AlgorithmError: 'Sampler job failed!'

In [33]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
try:
    backend = service.backend('ibm_brisbane')
    print(f"Brisbane status: {backend.status()}")
    print(f"Brisbane operational: {backend.status().operational}")
    print(f"Pending jobs: {backend.status().pending_jobs}")
except Exception as e:
    print(f"Cannot access Brisbane: {e}")

Brisbane status: <qiskit_ibm_runtime.models.backend_status.BackendStatus object at 0x0000020DC5224BC0>
Brisbane operational: True
Pending jobs: 1738


In [34]:
# List all available real quantum backends
service = QiskitRuntimeService()
real_backends = service.backends(simulator=False, operational=True)
print("Available real quantum backends:")
for backend in real_backends:
    print(f"- {backend.name}: {backend.status().operational}")

Available real quantum backends:
- ibm_brisbane: True
- ibm_torino: True


In [35]:
# Try a different real quantum backend
try:
    # Common IBM backends that might be available
    backend_names = ['ibm_kyoto', 'ibm_osaka', 'ibm_sherbrooke', 'ibm_torino']
    
    for backend_name in backend_names:
        try:
            backend = service.backend(backend_name)
            if backend.status().operational:
                print(f"Using {backend_name} instead")
                sampler = BackendSampler(backend)
                fidelity = ComputeUncompute(sampler=sampler)
                quantum_kernel = FidelityQuantumKernel(feature_map=feature_map, fidelity=fidelity)
                break
        except:
            continue
            
except Exception as e:
    print(f"Error accessing real backends: {e}")

Using ibm_torino instead


C:\Users\leqian\AppData\Local\Temp\ipykernel_14852\3217931965.py:11: DeprecationWarning: The class ``qiskit.primitives.backend_sampler.BackendSampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `BackendSampler` class is `BackendSamplerV2`.
  sampler = BackendSampler(backend)


In [36]:
# Test with the smallest available real backend
service = QiskitRuntimeService()
backends = service.backends(simulator=False, operational=True)
if backends:
    smallest_backend = min(backends, key=lambda b: b.configuration().n_qubits)
    print(f"Trying smallest real backend: {smallest_backend.name}")
    # Use this backend instead of Brisbane

Trying smallest real backend: ibm_brisbane


In [17]:
# Predict and evaluate
print("Making predictions...")
y_train_pred_quantum = qsvc.predict(X_train_pca[:,:n_features])
accuracy_train_quantum = accuracy_score(y_train, y_train_pred_quantum)

print(f"Quantum SVM Accuracy: {accuracy_train_quantum:.3f}")

y_pred_quantum = qsvc.predict(X_test_pca[:,:n_features])
accuracy_quantum = accuracy_score(y_test, y_pred_quantum)

print(f"Quantum SVM Accuracy: {accuracy_quantum:.3f}")

Making predictions...
Quantum SVM Accuracy: 0.712
Quantum SVM Accuracy: 0.620


In [14]:
def run_vqc(X_train, X_test, y_train, y_test, backend, service=None):
    """Run Variational Quantum Classifier as alternative"""
    print("Running Variational Quantum Classifier...")
    
    num_features = X_train.shape[1]
    
    try:
        # Feature map
        feature_map = ZZFeatureMap(num_features, reps=1)
        
        # Ansatz (parameterized quantum circuit)
        ansatz = TwoLocal(num_features, ['ry', 'rz'], 'cz', reps=1)
        
        # Create VQC
        if service:
            with Session(service=service, backend=backend) as session:
                sampler = Sampler(session=session)
                vqc = VQC(sampler=sampler,
                         feature_map=feature_map,
                         ansatz=ansatz,
                         optimizer='COBYLA',
                         max_iter=50)  # Reduced iterations for faster execution
        else:
            sampler = StatevectorSampler()
            vqc = VQC(sampler=sampler,
                     feature_map=feature_map,
                     ansatz=ansatz,
                     optimizer='COBYLA',
                     max_iter=50)
        
        # Train
        print("Training VQC...")
        vqc.fit(X_train, y_train)
        
        # Predict
        y_pred_vqc = vqc.predict(X_test)
        accuracy_vqc = accuracy_score(y_test, y_pred_vqc)
        
        print(f"VQC Accuracy: {accuracy_vqc:.3f}")
        return accuracy_vqc, y_pred_vqc
        
    except Exception as e:
        print(f"VQC failed: {e}")
        return None, None



In [63]:
# Feature map
feature_map = ZZFeatureMap(num_features, reps=1)

# Ansatz (parameterized quantum circuit)
ansatz = TwoLocal(num_features, ['ry', 'rz'], 'cz', reps=1)



In [66]:
# Create VQC
sampler = Sampler(mode="batch")
vqc = VQC(sampler=sampler,
            feature_map=feature_map,
            ansatz=ansatz,
            optimizer='COBYLA',
            max_iter=50)  # Reduced iterations for faster execution

# Train


C:\Users\leqian\AppData\Local\Temp\ipykernel_4856\87125657.py:2: DeprecationWarning: Passing a backend as a string is deprecated as of qiskit-ibm-runtime 0.26.0 and will be removed no sooner than 3 months after the release date. Use the actual backend object instead.
  sampler = Sampler(mode="batch")


QiskitBackendNotFoundError: 'No backend matches the criteria. Learn more about available backends here https://cloud.ibm.com/docs/quantum-computing?topic=quantum-computing-choose-backend '

In [ ]:
print("Training VQC...")
vqc.fit(X_train, y_train)

# Predict
y_pred_vqc = vqc.predict(X_test)
accuracy_vqc = accuracy_score(y_test, y_pred_vqc)

print(f"VQC Accuracy: {accuracy_vqc:.3f}")
    


In [61]:
# Run VQC as alternative
print("\n" + "="*50)
acc_vqc, pred_vqc = run_vqc(
    X_train_small, X_test_small, y_train_small, y_test_small, backend, service
)




Running Variational Quantum Classifier...


C:\Users\leqian\AppData\Local\Temp\ipykernel_4856\1348500438.py:16: DeprecationWarning: The service parameter is deprecated as of qiskit-ibm-runtime 0.26.0 and will be removed no sooner than 3 months after the release date. The service can be extracted from the backend object so it is no longer necessary.
  with Session(service=service, backend=backend) as session:


VQC failed: '400 Client Error: Bad Request for url: https://us-east.quantum-computing.cloud.ibm.com/sessions. {"errors":[{"code":1352,"message":"You are not authorized to run a session when using the open plan.","solution":"Create an instance of a different plan type or use a different execution mode <https://quantum.cloud.ibm.com/docs/guides/execution-modes>.","more_info":"https://cloud.ibm.com/apidocs/quantum-computing#error-handling"}],"trace":"d93e9f4c-51a3-4d51-871f-844df53ad0fb"}\n'


In [ ]:
# Summary
print("\n" + "="*50)
print("RESULTS SUMMARY:")
print(f"Classical SVM Accuracy: {acc_classical:.3f}")
if acc_quantum:
    print(f"Quantum SVM Accuracy:   {acc_quantum:.3f}")
if acc_vqc:
    print(f"VQC Accuracy:          {acc_vqc:.3f}")

# Print actual vs predicted for small test set
print(f"\nActual labels:    {y_test_small}")
print(f"Classical pred:   {pred_classical}")
if pred_quantum is not None:
    print(f"Quantum pred:     {pred_quantum}")
if pred_vqc is not None:
    print(f"VQC pred:         {pred_vqc}")



In [ ]:
# Installation requirements
print("Required packages:")
print("pip install qiskit qiskit-ibm-runtime qiskit-machine-learning")
print("pip install qiskit-aer scikit-learn pandas numpy matplotlib")
print("\nNote: qiskit-ibm-provider is deprecated, only use qiskit-ibm-runtime")
print("Important: Use FidelityQuantumKernel instead of QuantumKernel in recent versions")
print("Make sure qiskit-machine-learning version is >= 0.6")
print("\n" + "="*50)



# Instructions for running on IBM Quantum Cloud:
"""
1. Create IBM Quantum account at https://quantum-computing.ibm.com/
2. Get your API token from your account settings
3. Replace "YOUR_IBM_QUANTUM_TOKEN_HERE" with your actual token
4. Or save credentials locally using:
from qiskit_ibm_runtime import QiskitRuntimeService
QiskitRuntimeService.save_account(channel="ibm_quantum", token="your_token")

5. For the Pima Indian Diabetes dataset:
- Download from: https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database
- Place the CSV file in your working directory
- Uncomment the pandas.read_csv line in load_pima_data()

Note: This code uses the modern qiskit-ibm-runtime package only.
The old qiskit-ibm-provider is deprecated and removed from imports.

Current limitations due to quantum hardware constraints:
- Uses small subset of data (20 training, 10 test samples)
- Limited features (4) due to qubit availability
- Consider using dimensionality reduction (PCA) for larger datasets
- Use data encoding techniques like amplitude encoding for efficiency
- Consider hybrid classical-quantum approaches for practical applications
"""